In [ ]:
#THERE ARE TWO SEPARATE TARGET COLUMNS TO PREDICT, EITHER IN DATASET 1 AND 2

In [1]:
#import libraries
import numpy as np
import pandas as pd
import os

In [2]:
#import datasets
df1 = pd.read_csv("../data/dataset_1.csv")
df2 = pd.read_csv("../data/dataset_2.csv")

In [3]:
# dataset 1 info
print(df1.shape)
df1.head()

(50000, 301)


,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_292,var_293,var_294,var_295,var_296,var_297,var_298,var_299,var_300,target
0,0,0,0.0,0.00,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0
1,0,0,0.0,3.00,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0
2,0,0,0.0,5.88,0.0,0,0,0,0,0,...,0.0,0,0,3,0,0,0,0.0,67772.7216,0
3,0,0,0.0,14.10,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0
4,0,0,0.0,5.76,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0


In [4]:
substr = "tar"
for s in df2.columns:
    if substr in s:
        print(f"{s} contains {substr}")

target contains tar


In [ ]:
# dataset 2 info
print(df2.shape)
df2.head()

In [ ]:
df1 = df1.add_suffix('_feat_1')
df2 = df2.add_suffix('_feat_2')
print(df1.head)

In [ ]:
merged_df = pd.concat([df1, df2], axis=1)

In [ ]:
print(merged_df.info)

In [ ]:
# split merged dataset into (train/test, validate) = (80%, 20%)
merged_df, valMerged = np.split(merged_df.sample(frac=1, random_state=42), [int(.8*len(merged_df))])

In [ ]:
# exporting validation dataset to csv
valMerged.to_csv('valData.csv', encoding='utf-8')

In [ ]:
# showing dublicate columns in data frame
from fast_ml.utilities import display_all
from fast_ml.feature_selection import get_duplicate_features

# reset index after merge
merged_df.reset_index(drop=True, inplace=True)

# checks weather there are duplicate columns names
duplicates = list(set([x for x in list(merged_df.columns) if list(merged_df.columns).count(x) > 1]))
print(duplicates)

duplicate_features = get_duplicate_features(merged_df)
duplicate_features.head(10)

In [ ]:
# flaten dataframe with duplicate columns to list
duplicate_features_list = duplicate_features.query("Desc=='Duplicate Values'")['feature2'].to_list()
print(duplicate_features_list)

In [ ]:
# drops duplicate columns and shows difference to starting df
merged_df_clean = merged_df.copy(deep=True)
merged_df_clean.drop(columns = duplicate_features_list, inplace=True)
print('Shape of Dataset before dropping the duplicate values features: ', merged_df.shape)
print('Shape of Dataset after dropping the duplicate values features: ', merged_df_clean.shape)

In [ ]:
from fast_ml.feature_selection import get_constant_features

constant_features = get_constant_features(merged_df_clean)
constant_features.head(10)

In [ ]:
print("first row data = \n", constant_features.iloc[0], "\n number of rows = \n", len(constant_features))

In [ ]:
constant_features = constant_features.loc[(constant_features.Perc > 0.99)]
constant_features.head(10)
print(constant_features.shape)

In [ ]:
constant_features_list = constant_features["Var"].to_list()
print(constant_features_list)

In [ ]:
print("Dataset shape before dropping constant and quasi constant features: ", merged_df_clean.shape)
merged_df_clean.drop(columns = constant_features_list, inplace=True)
print("Dataset shape after dropping constant and quasi constant features: ", merged_df_clean.shape)

In [ ]:
# double check for other constant features
quasi_constant_features = get_constant_features(merged_df_clean, threshold=0.99, dropna=False)
quasi_constant_features.head(10)

In [ ]:
from sklearn import datasets

cor_matrix = merged_df_clean.corr().abs()
print(cor_matrix)

In [ ]:
# should be graph with corellation matrix but too big data
# import seaborn as sb
# sb.heatmap(cor_matrix, cmap="Blues", annot=True)

In [ ]:
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))
print(upper_tri)

In [ ]:
cols_to_drop = [column for column in upper_tri.columns if any(upper_tri[column]>0.95)]
print(cols_to_drop)

In [ ]:
# dropping columns with high corellation
merged_df_clean.drop(columns = list(cols_to_drop), axis=1, inplace=True)
print(merged_df_clean.shape)

In [ ]:
merged_df_clean.head(10)

In [ ]:
# Columns with correlation less than 0.001, would we drop this?

cols_to_drop = [column for column in upper_tri.columns if any(upper_tri[column]<0.001)]
print(len(cols_to_drop))

Description:
 - For each column, it first computes the Z-score of each value in the column, relative to the column mean and standard deviation.

- It then takes the absolute Z-score because the direction does not matter, only if it is below the threshold.

- all(axis=1) ensures that for each row, all column satisfy the constraint.

- Finally, the result of this condition is used to index the dataframe.

In [ ]:
# description above
from scipy import stats

merged_df_clean[(np.abs(stats.zscore(merged_df_clean)) < 3).all(axis=1)]

In [ ]:
print(merged_df_clean.shape)

In [ ]:
merged_df_clean.loc[:,['target_feat_1', 'target_feat_2']]

In [ ]:
substr = "tar"
for s in merged_df_clean.columns:
    if substr in s:
        print(f"{s} contains {substr}")

In [ ]:
# exporting preprocessed dataset to csv
merged_df_clean.to_csv('cleanData.csv', encoding='utf-8')